In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from datetime import datetime
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


2025-09-06 05:16:21.984791: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-06 05:16:22.061982: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-06 05:16:22.915841: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-06 05:16:26.291006: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [4]:
# Load the data
df = pd.read_csv("/home/astha/minor_pro/Network-Prediction-Analysis/src/data/Data1.csv")
df['PERIOD_START_TIME'] = pd.to_datetime(df['PERIOD_START_TIME'])
df.sort_values('PERIOD_START_TIME', inplace=True)

In [5]:
df.head()

,PERIOD_START_TIME,pl,pd,pdv
0,2022-07-04 06:00:00,0.003754,0.975612,0.018548
1,2022-07-04 06:15:00,0.003019,0.976934,0.019894
2,2022-07-04 06:30:00,0.005802,0.977411,0.020060
3,2022-07-04 06:45:00,0.004035,0.977569,0.021209
4,2022-07-04 07:00:00,0.005641,0.981013,0.024581


In [6]:
df.tail()

,PERIOD_START_TIME,pl,pd,pdv
5888,2022-10-01 22:45:00,0.004487,0.744733,0.044438
5889,2022-10-01 23:00:00,0.001543,0.732024,0.031747
5890,2022-10-01 23:15:00,0.001963,0.728793,0.028551
5891,2022-10-01 23:30:00,0.008098,0.738851,0.038595
5892,2022-10-01 23:45:00,0.006501,0.734670,0.036419


In [13]:
# Define features and sequence length
features = ['pl', 'pd', 'pdv']
sequence_length = 5
test_date = pd.to_datetime("2022-09-27").date()

# ---- 1. Normalize All Data ----
scalers = {}
scaled_data = []

for f in features:
    scaler = MinMaxScaler()

    scaled_col = scaler.fit_transform(df[[f]])

    scalers[f] = scaler

    scaled_data.append(scaled_col)
scaled_data = np.hstack(scaled_data)

In [14]:
# --- Create sequences ---
def create_sequences(data, seq_length):
    X, y = [], []

    for i in range(len(data) - seq_length):

        X.append(data[i : i + seq_length])

        y.append(data[i + seq_length])

    return np.array(X), np.array(y)




X_seq, y_seq = create_sequences(scaled_data, sequence_length)


In [15]:
print("PL Min:", df['pl'].min())
print("PL Max:", df['pl'].max())


PL Min: 0.0005765917323529
PL Max: 0.2220553330366414


In [16]:
# Separate into train and test sets
train_size = int(len(df[df['PERIOD_START_TIME'].dt.date < test_date]) * (1 - 0.2)) # Use a portion of data for training
X_train, y_train = X_seq[:train_size], y_seq[:train_size]
X_test, y_test = X_seq[train_size:], y_seq[train_size:]


In [17]:
# ---- 3. Build and Train LSTM Model with Callbacks ----
model = Sequential([
    LSTM(256, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.3),
    LSTM(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(3) # Output: pl, pd, pdv
])

model.compile(optimizer='adam', loss='mse')



2025-09-06 05:24:29.543249: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/astha/minor_pro/Network-Prediction-Analysis/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
# Callbacks for improved training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
]


In [19]:
# Use a portion of the training data as validation data
history = model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1,
                    validation_split=0.2, callbacks=callbacks)

Epoch 1/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - loss: 0.0225 - val_loss: 0.0047 - learning_rate: 0.0010
Epoch 2/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0071 - val_loss: 0.0028 - learning_rate: 0.0010
Epoch 3/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0060 - val_loss: 0.0033 - learning_rate: 0.0010
Epoch 4/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0054 - val_loss: 0.0023 - learning_rate: 0.0010
Epoch 5/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0049 - val_loss: 0.0021 - learning_rate: 0.0010
Epoch 6/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0047 - val_loss: 0.0020 - learning_rate: 0.0010
Epoch 7/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0044 - val_loss: 0.0022 - learning_rate: 0.0010
Epoch 8/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0043 - val_loss: 0.0024 - learning_rate: 0.0010
Epoch 9/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0041 - val_loss: 0.0019 - learning_rate: 

In [25]:
# ---- 4. Perform Rolling Prediction on Test Data ----
results = []
test_df = df[df['PERIOD_START_TIME'].dt.date == test_date].copy()

# The rolling window starts at the end of the training data
rolling_window = X_test[0].reshape(1, sequence_length, X_test.shape[2])

for i in range(len(test_df)):
    # Predict the next time step (one-step-ahead forecast)
    pred_scaled = model.predict(rolling_window, verbose=0)[0]

    # Update the rolling window with the predicted value
    rolling_window = np.vstack((rolling_window[0, 1:], pred_scaled)).reshape(1, sequence_length, X_test.shape[2])

    # Inverse transform to original scale
    pred_unscaled = np.zeros(3)
    for j, f in enumerate(features):
        pred_value_reshaped = pred_scaled[j].reshape(1, -1)
        pred_unscaled[j] = scalers[f].inverse_transform(pred_value_reshaped)[0][0]

    # Store the result
    results.append({
        "PERIOD_START_TIME": test_df['PERIOD_START_TIME'].iloc[i],
        "pl_lstm": pred_unscaled[0],
        "pd_lstm": pred_unscaled[1],
        "pdv_lstm": pred_unscaled[2]
    })

    # Update the rolling window with the *actual* next value from the test set
    actual_next_scaled = y_test[i].reshape(1, -1)
    rolling_window = np.vstack((rolling_window[0, 1:], actual_next_scaled)).reshape(1, sequence_length, X_test.shape[2])
    results.append({
        "PERIOD_START_TIME": test_df['PERIOD_START_TIME'].iloc[i],
        "pl_lstm": pred_unscaled[0],
        "pd_lstm": pred_unscaled[1],
        "pdv_lstm": pred_unscaled[2]
    })

    # Update the rolling window with the *actual* next value from the test set

    actual_next_scaled = y_test[i].reshape(1, -1)

    rolling_window = np.vstack((rolling_window[0, 1:], actual_next_scaled)).reshape(1, sequence_length, X_test.shape[2])




# --- Save results to CSV ---
result_df = pd.DataFrame(results)
output_file = '/home/astha/minor_pro/Network-Prediction-Analysis/src/data/lstm_result.csv'
result_df.to_csv(output_file, index=False)

print(f"Forecast results saved to lstm_reslt.csv")



Forecast results saved to lstm_reslt.csv


In [26]:
lstm = pd.read_csv('/home/astha/minor_pro/Network-Prediction-Analysis/src/data/lstm_result.csv')

In [27]:
lstm.head()

,PERIOD_START_TIME,pl_lstm,pd_lstm,pdv_lstm
0,2022-09-27 06:00:00,0.002509,0.793883,0.026535
1,2022-09-27 06:00:00,0.002509,0.793883,0.026535
2,2022-09-27 06:15:00,0.003919,0.777708,0.028238
3,2022-09-27 06:15:00,0.003919,0.777708,0.028238
4,2022-09-27 06:30:00,0.004487,0.778523,0.029425
